In [29]:
with open("22/input.txt") as f:
    data = f.read().splitlines()
data

['4,8,311~6,8,311',
 '2,7,275~5,7,275',
 '3,8,80~4,8,80',
 '7,6,12~7,8,12',
 '0,3,259~3,3,259',
 '4,6,197~4,9,197',
 '4,6,312~4,6,312',
 '7,2,65~7,2,66',
 '9,7,272~9,8,272',
 '4,4,230~4,4,230',
 '1,4,87~3,4,87',
 '3,6,137~6,6,137',
 '7,0,253~7,1,253',
 '1,4,270~1,4,270',
 '9,5,97~9,5,100',
 '5,5,199~5,5,199',
 '0,7,271~3,7,271',
 '1,1,299~1,3,299',
 '1,5,180~1,6,180',
 '8,6,2~8,6,4',
 '9,2,8~9,4,8',
 '8,1,44~8,3,44',
 '6,7,35~8,7,35',
 '3,1,234~3,2,234',
 '4,1,38~4,4,38',
 '8,6,16~9,6,16',
 '4,8,29~6,8,29',
 '1,2,25~2,2,25',
 '6,2,291~6,2,292',
 '7,9,230~9,9,230',
 '4,4,144~4,7,144',
 '7,7,261~9,7,261',
 '4,7,149~4,9,149',
 '0,8,246~2,8,246',
 '2,4,91~2,6,91',
 '6,7,15~8,7,15',
 '3,6,99~3,6,101',
 '5,2,214~8,2,214',
 '2,7,221~2,9,221',
 '4,8,284~6,8,284',
 '6,2,100~8,2,100',
 '1,6,196~4,6,196',
 '4,4,224~7,4,224',
 '2,5,63~2,5,65',
 '6,5,111~6,7,111',
 '1,7,86~1,9,86',
 '0,5,95~3,5,95',
 '0,4,309~3,4,309',
 '1,5,152~4,5,152',
 '1,5,96~3,5,96',
 '5,0,46~7,0,46',
 '1,5,312~1,7,312',
 '9,

In [30]:

from itertools import product

In [31]:
from dataclasses import dataclass
from collections import namedtuple


class BrickRange(tuple[int, int]):
    @property
    def start(self):
        return self[0]
    
    @property
    def end(self):
        return self[1]
    
    @property
    def range(self):
        return range(self.start, self.end+1)

@dataclass
class Brick:
    x: BrickRange
    y: BrickRange
    z: BrickRange

    @classmethod
    def from_tuples(cls, x: tuple[int, int], y: tuple[int, int], z: tuple[int, int]):
        return cls(BrickRange(x), BrickRange(y), BrickRange(z))
    
    def coordinates(self) -> set:
        return set(product(self.x.range, self.y.range, self.z.range))
    
    def collides(self, other):
        if isinstance(other, Brick):
            return len(self.coordinates().intersection(other.coordinates())) > 0
        raise ValueError(f"{other} is not Brick")
    
    def __repr__(self) -> str:
        return f"Brick({self.x}, {self.y}, {self.z})"

In [32]:
bricks: list[Brick] = []
for line in data:
    start, end = line.split("~")
    x_start, y_start, z_start = map(int, start.split(","))
    x_end, y_end, z_end = map(int, end.split(","))
    bricks.append(Brick.from_tuples((x_start, x_end), (y_start, y_end), (z_start, z_end)))
bricks

[Brick((4, 6), (8, 8), (311, 311)),
 Brick((2, 5), (7, 7), (275, 275)),
 Brick((3, 4), (8, 8), (80, 80)),
 Brick((7, 7), (6, 8), (12, 12)),
 Brick((0, 3), (3, 3), (259, 259)),
 Brick((4, 4), (6, 9), (197, 197)),
 Brick((4, 4), (6, 6), (312, 312)),
 Brick((7, 7), (2, 2), (65, 66)),
 Brick((9, 9), (7, 8), (272, 272)),
 Brick((4, 4), (4, 4), (230, 230)),
 Brick((1, 3), (4, 4), (87, 87)),
 Brick((3, 6), (6, 6), (137, 137)),
 Brick((7, 7), (0, 1), (253, 253)),
 Brick((1, 1), (4, 4), (270, 270)),
 Brick((9, 9), (5, 5), (97, 100)),
 Brick((5, 5), (5, 5), (199, 199)),
 Brick((0, 3), (7, 7), (271, 271)),
 Brick((1, 1), (1, 3), (299, 299)),
 Brick((1, 1), (5, 6), (180, 180)),
 Brick((8, 8), (6, 6), (2, 4)),
 Brick((9, 9), (2, 4), (8, 8)),
 Brick((8, 8), (1, 3), (44, 44)),
 Brick((6, 8), (7, 7), (35, 35)),
 Brick((3, 3), (1, 2), (234, 234)),
 Brick((4, 4), (1, 4), (38, 38)),
 Brick((8, 9), (6, 6), (16, 16)),
 Brick((4, 6), (8, 8), (29, 29)),
 Brick((1, 2), (2, 2), (25, 25)),
 Brick((6, 6), (2, 2)

In [42]:
def print_bricks(bricks: list[Brick], ignore: str = "y"):
    coords = ["x", "y", "z"]
    coords.remove(ignore)
    first, second = coords
    min_first, max_first = 0, max(getattr(b, first).end for b in bricks)
    min_second, max_second = 0, max(getattr(b, second).end for b in bricks)

    map = [["." for _ in range(max_first + 1)] for _ in range(max_second +1)]

    for i, brick in enumerate(bricks):
        first_start, first_end = getattr(brick, first)
        second_start, second_end = getattr(brick, second)
        for f in range(first_start, first_end + 1):
            for s in range(second_start, second_end + 1):
                map[s][f] = f"{i:>6}"
    return map

In [43]:
print_bricks(bricks, "y")[::-1]

[['.', '.', '.', '.', '.', '.', '   696', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '   813', '   813', '   813', '   813', '.'],
 ['.', '.', '.', '   635', '   635', '   635', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '  1293', '  1293', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '   948', '   800', '   800', '   800', '.', '.', '.'],
 ['   555', '   555', '   555', '.', '.', '.', '.', '.', '.', '.'],
 ['   942',
  '   942',
  '  1143',
  '   942',
  '   935',
  '   935',
  '.',
  '.',
  '.',
  '.'],
 ['.', '   565', '.', '  1095', '  1095', '  1071', '.', '.', '.', '.'],
 ['   578', '  1347', '  1347', '  1347', '.', '.', '.', '.', '.', '.'],
 ['.', '  1355', '  1176', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '   565', '.', '.', '.', '  1336', '  1369', '   614', '.', '.'],
 ['   982', '  1359', '.', '.', '.', '.', '.', '   614'

In [44]:
print_bricks(bricks, "x")[::-1]

[['.', '.', '.', '.', '.', '   696', '   696', '   696', '   696', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '   813', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '   635', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '  1293', '.', '.', '.', '.'],
 ['   800', '.', '.', '.', '   948', '   948', '   948', '   948', '.', '.'],
 ['.', '   555', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['  1143', '.', '   935', '.', '.', '.', '.', '   942', '.', '.'],
 ['  1095', '.', '.', '.', '.', '  1071', '.', '.', '.', '.'],
 ['  1347', '   578', '.', '.', '.', '   565', '.', '.', '.', '.'],
 ['.',
  '  1176',
  '  1355',
  '  1355',
  '  1355',
  '   565',
  '  1066',
  '  1066',
  '  1066',
  '.'],
 ['.', '.', '  1369', '  1369', '  1369', '  1336', '  1336', '.', '.', '.'],
 ['  1359',
  '  1359',
  '  1359',
  '  1359',
  '   982',
  '   982'

In [45]:
def can_fall(brick: Brick, bricks: list[Brick]) -> Brick | None:
    if brick.z.start > 1:
        old_start, old_end = brick.z
        new_start, new_end = old_start - 1, old_end - 1
        new_brick = Brick.from_tuples(brick.x, brick.y, (new_start, new_end))

        collides = any(new_brick.collides(other) for other in bricks if other != brick)
        if not collides:
            return new_brick
    return None

In [46]:
for i, brick in enumerate(bricks):
    if brick.z.start > 1:
        collides = False
        while brick.z.start != 1 and not collides:
            new_brick = can_fall(brick, bricks)
            if new_brick:
                brick = new_brick
            else:
                collides = True
        bricks[i] = brick

In [47]:
print_bricks(bricks)[::-1]

[['.', '.', '.', '.', '.', '.', '   696', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '   813', '   813', '   813', '   813', '.'],
 ['.', '.', '.', '   635', '   635', '   635', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '  1293', '  1293', '   800', '   800', '   800', '.', '.', '.'],
 ['   555', '   555', '  1143', '  1095', '  1095', '.', '.', '.', '.', '.'],
 ['   942',
  '   942',
  '   942',
  '   942',
  '   935',
  '   935',
  '.',
  '.',
  '.',
  '.'],
 ['.', '   565', '.', '.', '.', '  1071', '.', '.', '.', '.'],
 ['   578', '  1355', '  1347', '  1347', '.', '.', '.', '   614', '.', '.'],
 ['.', '   565', '.', '.', '.', '.', '.', '   614', '.', '.'],
 ['.', '.', '.', '   364', '.', '.', '.', '   614', '   924', '.'],
 ['   982',
  '  1359',
  '  1176',
  '   364',
  '     0',


In [48]:
from copy import copy

def find_safe_to_disintegrate(bricks: list[Brick]) -> list[Brick]:
    safe = []
    for brick in bricks:
        bricks_copy = copy(bricks)
        bricks_copy.remove(brick)
        if not any(can_fall(b, bricks_copy) for b in bricks_copy):
            safe.append(brick)
    return safe

In [49]:
find_safe_to_disintegrate(bricks)

[]

In [10]:
print_bricks(bricks, "x")[::-1]

[['.', 'G', '.'],
 ['.', 'G', '.'],
 ['.', 'F', '.'],
 ['E', 'E', 'E'],
 ['B', '.', 'C'],
 ['A', 'A', 'A'],
 ['.', '.', '.']]

In [11]:
print_bricks(bricks, "y")[::-1]

[['.', 'G', '.'],
 ['.', 'G', '.'],
 ['F', 'F', 'F'],
 ['D', '.', 'E'],
 ['C', 'C', 'C'],
 ['.', 'A', '.'],
 ['.', '.', '.']]